# 6. Import and Merge Data

In [1]:
import pandas as pd
import glob

# List all CSV files in the folder
file_paths = [
    r"C:\Users\Asus\Music\Citibike\202201-citibike-tripdata\202201-citibike-tripdata_1.csv",
    r"C:\Users\Asus\Music\Citibike\202201-citibike-tripdata\202201-citibike-tripdata_2.csv"
]

# Combine all files into one DataFrame
all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)

# Display the first few rows
print(all_data.head())

C:\Users\Asus\AppData\Local\Temp\ipykernel_7272\1565315616.py:11: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)


            ride_id  rideable_type               started_at  \
0  BFD29218AB271154  electric_bike  2022-01-21 13:13:43.392   
1  7C953F2FD7BE1302   classic_bike  2022-01-10 11:30:54.162   
2  95893ABD40CED4B8  electric_bike  2022-01-26 10:52:43.096   
3  F853B50772137378   classic_bike  2022-01-03 08:35:48.247   
4  7590ADF834797B4B   classic_bike  2022-01-22 14:14:23.043   

                  ended_at       start_station_name start_station_id  \
0  2022-01-21 13:22:31.463  West End Ave & W 107 St          7650.05   
1  2022-01-10 11:41:43.422             4 Ave & 3 St          4028.04   
2  2022-01-26 11:06:35.227          1 Ave & E 62 St          6753.08   
3  2022-01-03 09:10:50.475          2 Ave & E 96 St          7338.02   
4  2022-01-22 14:34:57.474          6 Ave & W 34 St          6364.10   

              end_station_name end_station_id  start_lat  start_lng  \
0  Mt Morris Park W & W 120 St        7685.14  40.802117 -73.968181   
1      Boerum Pl\t& Pacific St        4488.09 

# 7. Fetch Weather Data
Get an API Token from NOAA:

In [2]:
import requests

# NOAA API token
token = "IjlRiTQPicwBnfVgePvsmZGuSVzbRxZa" 
headers = {"token": token}

# Request weather data
url = "https://www.ncei.noaa.gov/access/services/data/v1"
params = {
    "dataset": "daily-summaries",
    "stations": "USW00094728",  # LaGuardia Airport
    "startDate": "2022-01-01",
    "endDate": "2022-12-31",
    "dataTypes": "TMAX,TMIN,PRCP",
    "format": "csv"
}
response = requests.get(url, headers=headers, params=params)

# Save to a CSV file
with open("weather_2022.csv", "wb") as f:
    f.write(response.content)

# 8. Merge CitiBike and Weather Data


In [4]:
print(all_data.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


In [8]:
import pandas as pd

# Load the weather data
weather = pd.read_csv("weather_2022.csv")

# Ensure 'DATE' column in weather is in datetime format
weather['DATE'] = pd.to_datetime(weather['DATE'])

# Ensure 'started_at' in all_data is converted to datetime
all_data['start_date'] = pd.to_datetime(all_data['started_at']).dt.normalize()

# Merge the datasets on the date
merged_data = pd.merge(
    all_data,
    weather,
    left_on='start_date',
    right_on='DATE',
    how='left'
)

# Save the merged dataset
merged_data.to_csv("merged_citibike_weather.csv", index=False)
print("Merged data saved!")

Merged data saved!
